# 2. Microsoft Foundry Agent - 작업 단계별 표시

이 노트북에서는 **Microsoft Foundry의 Agent Service**를 사용하여 AI 에이전트의 작업 과정을 단계별로 표시하는 방법을 보여줍니다.

## 주요 내용
- 🤖 **Microsoft Foundry Agent**: 에이전트 생성 및 실행
- 📋 **Run Steps**: 에이전트 실행 단계 추적 API
- 🔧 **도구 호출 표시**: Code Interpreter, File Search, Function 등
- 📊 **실시간 상태**: 스트리밍 이벤트로 진행 상황 표시

> 📖 참고: [Microsoft Foundry Agent Service 문서](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/)

---

## 2.1 환경 설정 및 라이브러리 임포트

In [ ]:
# 필요한 패키지 설치
%pip install --quiet azure-ai-projects azure-identity python-dotenv

In [ ]:
import os
import time
import json
from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown, clear_output

# Azure AI SDK
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

from azure.ai.projects.models import PromptAgentDefinition
from azure.ai.projects.models import CodeInterpreterTool
from azure.ai.projects.models import WebSearchPreviewTool, ApproximateLocation

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 로드 완료!")

In [ ]:
# Microsoft Foundry Project 클라이언트 설정
PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT", "")

if PROJECT_ENDPOINT:
    credential = DefaultAzureCredential()
    client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
    print("✅ Microsoft Foundry Project 클라이언트 연결 완료!")
    print(f"   엔드포인트: {PROJECT_ENDPOINT}")
else:
    client = None
    print("⚠️ AZURE_AI_PROJECT_ENDPOINT가 설정되지 않았습니다.")
    print("   .env 파일에 엔드포인트를 추가해주세요.")
    print("   예: https://<project-name>.services.ai.azure.com/api/projects/<project-id>")

## 2.2 WebSearch 에이전트 생성 및 테스트

In [ ]:
# 2.2 WebSearchAgent 생성
AGENT_NAME = "WebSearchAgent"
WEB_SEARCH_INSTRUCTIONS = """너는 Tools에 등록된 Web search 기반으로 답변하는 에이전트입니다.

중요 규칙:
1. 최신 정보가 필요한 질문에는 반드시 웹 검색을 사용하세요
2. 검색 결과를 기반으로 정확하고 최신의 정보를 제공하세요
3. 답변 시 출처 URL을 포함하세요
4. 여러 출처의 정보를 종합하여 균형잡힌 답변을 제공하세요
5. 검색 결과가 불충분하면 추가 검색을 수행하세요"""

try:
    definition = PromptAgentDefinition(
        model="gpt-4.1",
        instructions=WEB_SEARCH_INSTRUCTIONS,
        tools=[
          WebSearchPreviewTool()
        #   WebSearchPreviewTool(
        #     user_location=ApproximateLocation(country="GB", city="London", region="London")
        #   )
        ]
    )
    
    agent = client.agents.create(
        name=AGENT_NAME,
        definition=definition
    )
    
    print(f"✅ {AGENT_NAME} 생성 완료!")
    print(f"   ID: {agent.id}")
    print(f"   Name: {agent.name}")
    print(f"   Model: {definition.model}")
    print(f"   Tools: Web Search")
    
except Exception as e:
    print(f"⚠️ 에이전트 생성 실패: {e}")
    print("\n💡 해결 방법:")
    print("   1. 'gpt-4.1' 모델이 배포되었는지 확인")
    print("   2. Web Search가 프로젝트에서 활성화되었는지 확인")

### 함수 호출 테스트

In [ ]:
try:
    print("⏳ WebSearchAgent 실행 중 (웹 검색 중...)")
    print(f"📍 Agent Name: {AGENT_NAME}")
    
    
    # SDK v2 - project client를 통해 OpenAI client 획듍
    openai_client = client.get_openai_client()
    
    # Conversation 생성
    conversation = openai_client.conversations.create()
    print(f"💬 Conversation ID: {conversation.id}")
    
    # Responses API 호출
    response = openai_client.responses.create(
        conversation=conversation.id,
        input="오늘 서울 날씨는 어떤가요?",
        extra_body={"agent": {"name": AGENT_NAME, "type": "agent_reference"}},
    )
    
    print("\n" + "=" * 80)
    print(f"🌐 {AGENT_NAME} 응답:")
    print("=" * 80)
    print(response.output_text)
    
    # Conversation 정리
    openai_client.conversations.delete(conversation_id=conversation.id)
    print("\n✅ 웹 검색 도구가 실시간 정보를 가져왔습니다!")
    
except NameError:
    print("⚠️ agent 또는 client가 정의되지 않았습니다.")
    print(f"💡 위의 환경 설정 및 {AGENT_NAME} 생성 셀을 먼저 실행하세요.")
    
except Exception as e:
    print(f"⚠️ 테스트 실패: {e}")
    import traceback
    print(f"\n상세 오류:\n{traceback.format_exc()}")

### 스트리밍 방식 함수 호출 테스트

In [ ]:
# Conversation 생성
conversation = openai_client.conversations.create()
print(f"💬 Conversation ID: {conversation.id}")
print()
    
stream_response = openai_client.responses.create(
    stream=True,
    conversation=conversation.id,
    input="오늘 서울 날씨는 어떤가요?",
    extra_body={"agent": {"name": AGENT_NAME, "type": "agent_reference"}},
)

print("🤖 응답: ", end="", flush=True)

for event in stream_response:
    if event.type == "response.created":
        pass  # 시작 시 별도 출력 불필요
    elif event.type == "response.output_text.delta":
        # 각 delta를 연결하여 출력 (줄바꿈 없이)
        print(event.delta, end="", flush=True)
    elif event.type == "response.output_item.done":
        if event.item.type == "message":
            item = event.item
            if item.content[-1].type == "output_text":
                text_content = item.content[-1]
                # URL 인용 출력
                if text_content.annotations:
                    print("\n\n📚 출처:")
                    for annotation in text_content.annotations:
                        if annotation.type == "url_citation":
                            print(f"   🔗 {annotation.url}")
    elif event.type == "response.completed":
        print("\n\n✅ 응답 완료!")

## 2.3 진행 상황 표시 함수

에이전트 실행 단계를 시각적으로 표시하는 간단한 헬퍼 함수들입니다.

In [ ]:
# 단계 유형별 아이콘
STEP_ICONS = {
    "thread_created": "📝",
    "message_added": "💬",
    "thinking": "🧠",
    "tool_calls": "🔧",
    "code_interpreter": "💻",
    "file_search": "📂",
    "function_call": "⚡",
    "message_creation": "✍️",
    "completed": "✅",
    "error": "❌",
}


def display_steps(steps: list, title: str = "Microsoft Foundry Agent 실행 중", is_complete: bool = False):
    """
    단계 목록을 HTML로 표시
    
    Args:
        steps: [{"type": str, "text": str, "status": str, "time": float}, ...]
        title: 표시할 제목
        is_complete: 완료 여부
    """
    if is_complete:
        bg_color = "rgba(76, 175, 80, 0.15)"
        border_color = "rgba(76, 175, 80, 0.4)"
        title_color = "#81c784"
    else:
        bg_color = "rgba(33, 150, 243, 0.15)"
        border_color = "rgba(33, 150, 243, 0.4)"
        title_color = "#64b5f6"
    
    html = f"""
    <div style="font-family: 'Segoe UI', sans-serif; padding: 15px; background: {bg_color}; 
                border: 1px solid {border_color}; border-radius: 10px; max-width: 650px;">
        <div style="font-weight: bold; color: {title_color}; margin-bottom: 12px; font-size: 1.1em;">
            🤖 {title}
        </div>
        <div style="border-left: 3px solid #0078d4; padding-left: 12px;">
    """
    
    for step in steps:
        step_type = step.get("type", "")
        icon = STEP_ICONS.get(step_type, "📌")
        status = step.get("status", "running")
        
        # 상태별 색상
        if status == "completed":
            status_icon = "✅"
            text_color = "#81c784"
        elif status == "running":
            status_icon = "⏳"
            text_color = "#64b5f6"
        elif status == "error":
            status_icon = "❌"
            text_color = "#ef5350"
        else:
            status_icon = "⚪"
            text_color = "#9e9e9e"
        
        time_str = f"+{step.get('time', 0):.1f}s" if step.get('time') else ""
        
        html += f"""
        <div style="padding: 6px 0; color: {text_color};">
            {status_icon} {icon} {step.get("text", "")}
            <span style="color: #bdbdbd; font-size: 0.85em; float: right;">{time_str}</span>
        </div>
        """
    
    html += "</div></div>"
    display(HTML(html))


def display_final_response(response_text: str, total_steps: int, total_time: float):
    """최종 응답을 표시"""
    html = f"""
    <div style="font-family: 'Segoe UI', sans-serif; padding: 15px; background: rgba(76, 175, 80, 0.15); 
                border: 1px solid rgba(76, 175, 80, 0.4); border-radius: 10px; max-width: 650px; margin-top: 10px;">
        <div style="font-weight: bold; color: #81c784; margin-bottom: 8px;">
            ✅ 작업 완료!
        </div>
        <div style="color: #e0e0e0; font-size: 0.9em;">
            📊 총 {total_steps}개 단계 | ⏱️ {total_time:.2f}초
        </div>
    </div>
    """
    display(HTML(html))
    display(Markdown(response_text))


print("✅ 진행 상황 표시 함수 정의 완료!")

## 2.4 WebSearch Agent 실행 함수

앞서 생성한 WebSearchAgent를 활용하여 진행 상황을 표시하는 함수들입니다.

### Responses API 구조 및 스트리밍 이벤트

`stream=True` 옵션으로 실시간 이벤트를 수신하며, 각 에이전트 작업 단계를 시각적으로 표시합니다.

```
openai_client.responses.create()
 └── Stream Events
      ├── response.created: 응답 시작
      ├── response.output_item.added: 함수 호출 시작 (웹 검색 등)
      ├── response.output_item.done: 함수 호출/메시지 완료 (annotations 포함)
      ├── response.output_text.delta: 텍스트 토큰 스트리밍
      └── response.completed: 응답 완료
```

### 웹 검색 에이전트 작업 단계

| 단계 | 이벤트 | 설명 |
|------|--------|------|
| 1️⃣ 준비 | - | OpenAI Client 및 Conversation 생성 |
| 2️⃣ 웹 검색 시작 | `response.created` | 에이전트가 웹 검색 도구 호출 |
| 3️⃣ 검색 결과 수신 | `response.output_item.done (function_call)` | 웹 검색 결과 반환 |
| 4️⃣ 응답 생성 | `response.output_text.delta` | 검색 결과 기반 응답 작성 |
| 5️⃣ 완료 | `response.completed` | 모든 작업 완료 |

**참고 문서:**
- [OpenAI Responses API Streaming](https://platform.openai.com/docs/api-reference/responses-streaming)
- [Microsoft Foundry Web Search Tool](https://learn.microsoft.com/ko-kr/azure/ai-foundry/agents/how-to/tools/web-search?view=foundry&pivots=python)

In [21]:
def run_websearch_agent_with_steps(user_message: str, show_sources: bool = True):
    """
    스트리밍 API로 WebSearchAgent 실행하며 각 단계를 상세히 표시
    
    Args:
        user_message: 사용자 질문
        show_sources: 출처 URL 표시 여부
    
    Returns:
        전체 응답 텍스트
    """
    if not client:
        print("⚠️ client 또는 agent가 설정되지 않았습니다.")
        return None
    
    steps = []
    start_time = time.time()
    full_response = ""
    sources = []
    web_search_started = False
    response_started = False
    
    def add_step(step_type: str, text: str, status: str = "running"):
        steps.append({
            "type": step_type,
            "text": text,
            "status": status,
            "time": time.time() - start_time
        })
        clear_output(wait=True)
        display_steps(steps, title="🌐 WebSearch Agent 실행 중")
    
    def complete_last_step(new_text: str = None):
        if steps:
            steps[-1]["status"] = "completed"
            if new_text:
                steps[-1]["text"] = new_text
            clear_output(wait=True)
            display_steps(steps, title="🌐 WebSearch Agent 실행 중")
    
    try:
        # 1. 준비 단계
        add_step("thinking", "OpenAI Client 준비 중...")
        openai_client = client.get_openai_client()
        complete_last_step("OpenAI Client 준비 완료")
        
        # 2. Conversation 생성
        add_step("thread_created", "Conversation 생성 중...")
        conversation = openai_client.conversations.create()
        complete_last_step(f"Conversation 생성 완료")
        
        # 3. 요청 전송
        add_step("message_added", f"질문 전송: '{user_message[:30]}...' " if len(user_message) > 30 else f"질문 전송: '{user_message}'")
        
        # 스트리밍 응답 요청
        stream_response = openai_client.responses.create(
            stream=True,
            conversation=conversation.id,
            input=user_message,
            extra_body={"agent": {"name": AGENT_NAME, "type": "agent_reference"}},
        )
        complete_last_step("질문 전송 완료")
        
        # 4. 스트리밍 이벤트 처리
        for event in stream_response:
            # 응답 시작 - 웹 검색 시작으로 간주
            if event.type == "response.created":
                if not web_search_started:
                    add_step("tool_calls", "🔍 웹 검색 시작...")
                    web_search_started = True
            
            # 웹 검색 함수 호출 감지
            elif event.type == "response.output_item.added":
                if hasattr(event, 'item'):
                    item_type = getattr(event.item, 'type', '')
                    if item_type == "function_call":
                        func_name = getattr(event.item, 'name', 'unknown')
                        if 'search' in func_name.lower() or 'web' in func_name.lower():
                            complete_last_step("🔍 웹 검색 시작")
                            add_step("tool_calls", "🌐 검색 쿼리 실행 중...")
            
            # 함수 호출 완료 (웹 검색 결과 수신)
            elif event.type == "response.output_item.done":
                if hasattr(event, 'item'):
                    item_type = getattr(event.item, 'type', '')
                    
                    # 함수 호출 완료 = 웹 검색 결과 수신
                    if item_type == "function_call":
                        complete_last_step("🌐 웹 검색 완료")
                        add_step("file_search", "📄 검색 결과 분석 중...")
                    
                    # 함수 호출 결과 처리
                    elif item_type == "function_call_output":
                        complete_last_step("📄 검색 결과 분석 완료")
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    
                    # 메시지 완료 - URL 인용 수집
                    elif item_type == "message":
                        for content in event.item.content:
                            if content.type == "output_text" and content.annotations:
                                for annotation in content.annotations:
                                    if annotation.type == "url_citation":
                                        sources.append(annotation.url)
            
            # 텍스트 스트리밍 시작
            elif event.type == "response.output_text.delta":
                if not response_started:
                    # 웹 검색이 시작되지 않았다면 (직접 응답하는 경우)
                    if web_search_started and len(steps) > 0 and steps[-1]["type"] == "tool_calls":
                        complete_last_step("🌐 웹 검색 결과 수신 완료")
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    elif not any(s["type"] == "message_creation" for s in steps):
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    response_started = True
                
                full_response += event.delta
            
            # 응답 완료
            elif event.type == "response.completed":
                complete_last_step("✍️ 응답 생성 완료")
                add_step("completed", "✅ 모든 작업 완료!", status="completed")
        
        # 최종 결과 표시
        total_time = time.time() - start_time
        clear_output(wait=True)
        display_steps(steps, title="🌐 WebSearch Agent 완료", is_complete=True)
        display_final_response(full_response, len(steps), total_time)
        
        # 출처 표시
        if show_sources and sources:
            print("\n📚 참고한 출처:")
            for i, url in enumerate(sources, 1):
                print(f"   {i}. 🔗 {url}")
        
        # Conversation 정리
        openai_client.conversations.delete(conversation_id=conversation.id)
        
        return full_response
        
    except Exception as e:
        add_step("error", f"오류 발생: {str(e)}", status="error")
        raise


print("✅ run_websearch_agent_with_steps() 함수 정의 완료!")
print("   사용법: run_websearch_agent_with_steps('오늘 서울 날씨는?')")

✅ run_websearch_agent_with_steps() 함수 정의 완료!
   사용법: run_websearch_agent_with_steps('오늘 서울 날씨는?')


In [22]:
# 상세 단계별 스트리밍 실행 예제
response = run_websearch_agent_with_steps("2024년 가장 인기있는 프로그래밍 언어는?")

2024년 가장 인기 있는 프로그래밍 언어에 대한 여러 글로벌 인기 지수(TIOBE, PYPL, Stack Overflow 등)와 국내 IT 업계 동향을 종합해보면, 인기 언어 TOP 5는 다음과 같습니다:

1. **Python(파이썬)**
   - 데이터 과학, 인공지능, 웹 개발, 자동화 등 다양한 분야에서 압도적인 1위를 차지하며 초보자와 전문가 모두에게 인기가 높습니다. 문법이 간단하고, 교육 및 산업계에서 폭넓게 사용됩니다[네이버 블로그](https://blog.naver.com/PostView.naver?blogId=bestaicoach&logNo=223791927303), [Turinglab 블로그](https://blog.naver.com/PostView.naver?blogId=turinglab&logNo=223690010557).

2. **JavaScript(자바스크립트)**
   - 웹 프론트엔드뿐 아니라 Node.js 등 서버 개발, 모바일 앱 개발까지 폭넓게 활용되며, Stack Overflow 개발자 설문에서는 11년 연속 1위에 오를 정도로 웹 생태계에서 필수적인 언어입니다[코드랩스 아카데미](https://codelabsacademy.com/ko/%EB%B8%94%EB%A1%9C%EA%B7%B8/2024s-most-in-demand-programming-languages-for-developers).

3. **Java(자바)**
   - 안정적인 산업 표준 언어로, 기업용 백엔드, 모바일(Android), 금융, 대용량 시스템 분야에서 꾸준히 사용됩니다. TIOBE와 PYPL 모두 상위권을 유지하고 있습니다[네이버 블로그](https://blog.naver.com/PostView.naver?blogId=turinglab&logNo=223690010557).

4. **C++(씨플플)**
   - 게임 엔진, 시스템 소프트웨어, 임베디드, 성능 중심의 프로그래밍 분야에서 지속적으로 높은 순위를 유지합니다[네이버 블로그](https://blog.naver.com/PostView.naver?blogId=turinglab&logNo=223690010557).

5. **Go(고)**
   - 최근 클라우드 네이티브, 서버 백엔드, 확장성 있는 인프라 개발 분야에서 인기가 크게 상승하며, 2024년 TIOBE 순위에서 7위, PYPL 순위에서도 꾸준한 성장세를 보입니다[네이버 블로그](https://blog.naver.com/PostView.naver?blogId=bestaicoach&logNo=223791927303).

추가로, 활용 분야의 특성을 반영해 **SQL**, **C#**, **TypeScript**, **Rust** 등이 상위권에 포함되고, **Kotlin**은 모바일(android) 분야와 모던 백엔드에서 점차 존재감을 드러내는 추세입니다[원티드](https://www.wanted.co.kr/events/article_240408a).

**참고자료**
- [2024–2025 TIOBE 프로그래밍 언어 순위 및 사용률 비교](https://blog.naver.com/PostView.naver?blogId=bestaicoach&logNo=223791927303)
- [2024년 12월 최신 프로그래밍 언어 순위](https://untitledtblog.tistory.com/203)
- [2024년 개발자를 위한 가장 수요가 많은 프로그래밍 언어](https://codelabsacademy.com/ko/%EB%B8%94%EB%A1%9C%EA%B7%B8/2024s-most-in-demand-programming-languages-for-developers)
- [프로그래밍 언어 순위: 프로그래밍 언어 2024년 12월](https://blog.naver.com/PostView.naver?blogId=turinglab&logNo=223690010557)


📚 참고한 출처:
   1. 🔗 https://untitledtblog.tistory.com/203
   2. 🔗 https://blog.naver.com/PostView.naver?blogId=bestaicoach&logNo=223791927303
   3. 🔗 https://codelabsacademy.com/ko/%EB%B8%94%EB%A1%9C%EA%B7%B8/2024s-most-in-demand-programming-languages-for-developers
   4. 🔗 https://blog.naver.com/PostView.naver?blogId=turinglab&logNo=223690010557
   5. 🔗 https://www.wanted.co.kr/events/article_240408a


## 📋 요약

### 구현된 기능

| 구성 요소 | 설명 |
|-----------|------|
| **WebSearchAgent** | Microsoft Foundry Agent Service 기반 웹 검색 에이전트 |
| **진행 상황 시각화** | HTML 기반 단계별 작업 과정 표시 |
| **실행 함수** | `run_websearch_agent()` - 간단한 실행<br>`run_websearch_agent_with_steps()` - 상세 단계 표시 |

---

### 에이전트 작업 흐름

```
┌─────────────────────────────────────────────────────────┐
│  1️⃣  OpenAI Client 준비 완료                            │
│  2️⃣  Conversation 생성 완료                             │
│  3️⃣  질문 전송 완료                                     │
│  4️⃣  🔍 웹 검색 시작                                    │
│  5️⃣  🌐 웹 검색 완료                                    │
│  6️⃣  📄 검색 결과 분석 완료                             │
│  7️⃣  ✍️  응답 생성 완료                                 │
│  8️⃣  ✅ 모든 작업 완료!                                 │
└─────────────────────────────────────────────────────────┘
```

---

### 스트리밍 이벤트

| 이벤트 | 설명 |
|--------|------|
| `response.created` | 응답 시작 |
| `response.output_item.added` | 함수 호출 시작 (웹 검색 등) |
| `response.output_item.done` | 함수 호출 완료 / 메시지 완료 |
| `response.output_text.delta` | 응답 텍스트 토큰 스트리밍 |
| `response.completed` | 모든 작업 완료 |

---

### 핵심 API

```python
# 1. 클라이언트 설정
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
openai_client = client.get_openai_client()

# 2. WebSearch 에이전트 생성
definition = PromptAgentDefinition(
    model="gpt-4.1",
    instructions="...",
    tools=[WebSearchPreviewTool()]
)
agent = client.agents.create(name="WebSearchAgent", definition=definition)

# 3. Conversation 생성
conversation = openai_client.conversations.create()

# 4. 스트리밍 호출
stream = openai_client.responses.create(
    stream=True,
    conversation=conversation.id,
    input="질문",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}}
)

# 5. 이벤트 처리
for event in stream:
    match event.type:
        case "response.created":
            print("🔍 웹 검색 시작...")
        case "response.output_item.done":
            print("🌐 검색 완료")
        case "response.output_text.delta":
            print(event.delta, end="", flush=True)
        case "response.completed":
            print("✅ 완료!")
```

---

**다음 노트북** → Semantic Kernel 멀티 에이전트 워크플로우